In [1]:
import os

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [2]:
import numpy as np
import pandas as pd
from src.features.resnet152 import get_resnet152_features
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.decomposition import PCA

/home/seara/Desktop/Github/sneakers-ml/src/features/resnet152.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
features, classes, class_to_idx = get_resnet152_features("data/merged/images/by-brands", save=True)

  0%|          | 0/93 [00:00<?, ?it/s]